In [86]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Concatenate , Add, Dot, Activation, Lambda
from tensorflow.keras.models import Model

from tensorflow.image import flip_up_down, flip_left_right, rot90
from tensorflow.linalg import normalize

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import math

import matplotlib.pyplot as plt
import sys 

In [87]:
from tensorflow.python.client import device_lib
#tf.disable_v2_behavior()

#print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(tf.__version__ )

Num GPUs Available:  1
2.9.1


## Sorted Filters

In this experiment, I will try creating a new convolutional layer. Each filter F will be decomposed into Fs and Fa, but only Fs will be learned. Fa will be preinitialised to represent an angle from $0..2\pi$.

Knowing that an antisymetric matrix $F_a$ can be parameterised by 4 values : 

$$F_a = \begin{bmatrix}a & b & c\\
d & 0 & -d \\
-c & -b & -a
\end{bmatrix}$$

I can use Sobel filters (G)  to calulate the dominant angle of $F_a$.

$$G_x = \begin{bmatrix}1& 0 & -1\\
2 & 0 & -2 \\
1 & 0 & -1
\end{bmatrix} * F_a \ \text{   and    } \ G_y = \begin{bmatrix}1& 2 & 1\\
0 & 0 & 0 \\
-1 & -2 & -1
\end{bmatrix} * F_a $$

Which can be writen as:

$$ G_x = 2a - 2c + 4d $$
$$ G_y = 2a+4b+2c$$

And knowing that the Sobel angle $\theta$ is 


$$ \theta = \tan^{-1} \left(\frac{G_y}{G_x} \right) =  \tan^{-1} \left(\frac{a+2b+c}{a-c+2d} \right) $$

I can then calculate a value of $d$ in such a way that for any given value of $a, b, c$, the Sobel angle of $F_a$ is $\theta$

$$d = \frac{1}{2}\cot(\theta)(a + 2b+c)-a+c$$

Knowing this identity, I can set $a, b$ and $c$ to random values, and ensure the dominant angle of $F_a$ simply by calculating $d$

Once $a, b, c$ and $d$ are found, I will aditionaly normalize them so that their sum is 1  (https://legacy.imagemagick.org/Usage/convolve/#zero-summing_normalization

To ensure that I only learn a symetric filter $F_s$, I will make use of the fact that $F_s$ can be parameterised by 3 values : 

$$F_s = \begin{bmatrix}\alpha & \beta & \alpha\\
 \beta & \gamma &  \beta \\
\alpha &  \beta & \alpha
\end{bmatrix}$$

So, by simply learning $\alpha,\beta$ and $\gamma$, I will force the $F_s$ to remain symetric


In [215]:
class SortedConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3,3), padding = 'VALID', strides = (1, 1), activation=None, use_bias = True):
        super(SortedConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation
        self.padding = padding
        self.kernel_initializer = tf.keras.initializers.GlorotNormal(seed=5)
        self.param_initializer =tf.keras.initializers.GlorotNormal(seed=5)

        self.bias_initializer = tf.initializers.Zeros()
        self.strides = strides
        self.use_bias = use_bias
        self.w = None
        self.bias = None

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "padding": self.padding,
            "strides": self.strides,
            "activation": self.activation,
            "use_bias": self.use_bias,
        })
        return config

    def get_weights(self):
        return self.w

    def build(self, input_shape):
        *_, n_channels = input_shape

        #####  BUILD Fa   ##### 

        a = tf.Variable(
            initial_value=self.param_initializer(shape=( 1,
                                                         n_channels,
                                                         self.filters),
                                 dtype='float32'), trainable=False)
        b = tf.Variable(
            initial_value=self.param_initializer(shape=( 1,
                                                         n_channels,
                                                         self.filters),
                                 dtype='float32'), trainable=False)
        c = tf.Variable(
            initial_value=self.param_initializer(shape=( 1,
                                                         n_channels,
                                                         self.filters),
                                 dtype='float32'), trainable=False)  

        d = 0.5 * (tf.multiply(1/tf.math.tan(tf.linspace(0.0, 2.0*math.pi, self.filters, axis=0)) , (a+2*b+c)) - a + c)
        
        self.w = tf.Variable(
            initial_value= tf.stack([tf.concat([a,b,c], axis=0) , 
                                    tf.concat( [d,tf.zeros([1, n_channels, self.filters]), -d], axis=0),
                                    tf.concat( [-c, -b, -a], axis=0)])  , trainable=False)
        


        if self.use_bias:
            self.bias = tf.Variable(
                initial_value=self.bias_initializer(shape=(self.filters,), 
                                                    dtype='float32'),
                trainable=True)


    def call(self, inputs, training=None):

        x =  tf.nn.conv2d(inputs, filters=self.w, strides=self.strides, 
                          padding=self.padding)



        
        if self.use_bias:
            #x_sym = x_sym + self.b_sym
            #x_anti = x_anti + self.b_anti
            x = x + self.bias

        return x
layer_a = SortedConv2D(64)


In [92]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.ndimage import rotate
import imutils

#https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/

def load_img(img_path,img_shape):
    
    img_rows=img_shape[0]
    img_cols=img_shape[1]
    #num_channel=img_shape[2]

    img = image.load_img(img_path , target_size=(img_rows, img_cols))    
    img = image.img_to_array(img) 

    img = np.expand_dims(img, axis=0)


    return img


In [219]:
img  = load_img('test_images/piano_zoom.png', img_shape=(224,224))


layer_a(img)
print(layer_a.get_weights().shape)
print(layer_a.get_weights()[:,:,0,31])

(3, 3, 3, 64)
tf.Tensor(
[[-3.3250183e-02  2.2738786e-01 -7.2154668e-05]
 [-4.2057171e+00  0.0000000e+00  4.2057171e+00]
 [ 7.2154668e-05 -2.2738786e-01  3.3250183e-02]], shape=(3, 3), dtype=float32)


In [184]:
a = tf.Variable(
    initial_value= tf.keras.initializers.Constant(value=2)(shape=(1,
                                                    3,
                                                    5),
                            dtype='float32'), trainable=False)

In [185]:
a

<tf.Variable 'Variable:0' shape=(1, 3, 5) dtype=float32, numpy=
array([[[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]]], dtype=float32)>

In [188]:
tf.linspace(360/5, 360, 5, axis=0)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 72., 144., 216., 288., 360.], dtype=float32)>

In [187]:
tf.multiply(tf.linspace(360/5, 360, 5, axis=0) , a)

<tf.Tensor: shape=(1, 3, 5), dtype=float32, numpy=
array([[[144., 288., 432., 576., 720.],
        [144., 288., 432., 576., 720.],
        [144., 288., 432., 576., 720.]]], dtype=float32)>

In [178]:
x = tf.stack([tf.concat([a,a,a], axis=0), tf.concat([a,a,a], axis=0), tf.concat([a,a,a], axis=0)])
x.shape

TensorShape([3, 3, 3, 2])

In [174]:
tf.linspace(360/64, 360, 64, axis=0)[32] 

<tf.Tensor: shape=(), dtype=float32, numpy=185.625>

In [208]:
tf.linspace((2*math.pi)/64, 2*math.pi, 64, axis=0)[31]

<tf.Tensor: shape=(), dtype=float32, numpy=3.1415927>

In [221]:
0.5* ((1/tf.math.tan(tf.linspace(0., 2.*math.pi, 64, axis=0))[31]) * (-0.29142228 + 2*0.02473969 +0.17145856 ) + 0.29142228 +0.17145856)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9375836>

In [227]:
tf.linspace(0., 2.*math.pi, 64, axis=0)[31]

<tf.Tensor: shape=(), dtype=float32, numpy=3.091726>